# Bisulfite 
- https://en.wikipedia.org/wiki/Bisulfite_sequencing
- Figure 2: Outline of the chemical reaction that underlies the bisulfite-catalyzed conversion of cytosine to uracil.
<img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Bisulfite_conversion.svg" />

In [ ]:
library(methylKit)
library(DSS)
library(edgeR)
library(tidyverse)
library(ggrepel)
library(patchwork)
library(genomation)
library(GenomicFeatures)
library(ORFik)
library(ggExtra)

In [ ]:
source('../scripts/util.R')

### Correlate with RNA-seq profiles

In [ ]:
> load("rna.RData")
> dim(y_rna)
> haveME <- row.names(y_rna) %in% row.names(ypr)
> y_rna <- y_rna[haveME,]
> dim(y_rna)
> fitrna <- glmFit(y_rna)
> Contrastrna <- makeContrasts(L.virgin-B.virgin, levels=y_rna$design)
> lrtrna <- glmLRT(fitrna, contrast=Contrastrna)
# Correlate methylation and expression
> topME$logFC.RNA <- lrtrna$table[row.names(topME),"logFC"]
> topME[1:30,c("Symbol","logFC","logFC.RNA")]

> plot(topME$logFC, topME$logFC.RNA, main="Lumina vs Basal",
+      xlab="Methylation logFC", ylab="Expression logFC",
+      pch=16, cex=0.8, col="gray30")
> abline(h=0, v=0, col="gray10", lty=2, lwd=2)


> Figure 8. Scatter plot of the log-fold-changes of methylation levels in gene promoters (x-axis) vs the log fold-changes of gene expression (y-axis).


In [ ]:
> RNAvsME <- lm(topME$logFC.RNA ~ 0 + topME$logFC)
> coef(summary(RNAvsME))

> abline(RNAvsME, col="red", lwd=2) 


In [ ]:
# Gene set testing
> ME <- data.frame(ID=row.names(topME), logFC=topME$logFC, stringsAsFactors=FALSE)
> inRNA <- ME$ID %in% row.names(y_rna)
> ME <- ME[inRNA,]
> fry(y_rna, index=ME, contrast=Contrastrna)

> logFC.ME <- rep_len(0,nrow(y_rna))
> names(logFC.ME) <- row.names(y_rna)
> logFC.ME[ME$ID] <- ME$logFC
> barcodeplot(lrtrna$table$logFC, gene.weights=logFC.ME,
+               labels=c("Basal","Luminal"), main="Luminal vs Basal",
+               xlab="Expression logFC", weights.label="Me logFC")
> legend("topright", col=c("red","blue"), lty=1, lwd=2,
+         legend=c("Hypermethylated in Luminal", "Hypomethylated in Luminal"))


### Focus on gene lists

In [ ]:
smaller_volcano <- function(genes,title){
    v = myDiff_tiles_ann[myDiff_tiles_ann$symbol %in% genes,] %>% 
        plot_hyperhypo(5,0.05,title=title,x_lim=c(-100,100),y_lim=7) +
        geom_point(
            data = res_promoters[res_promoters$symbol %in% genes,] %>% filter(
                    abs(meth.diff) >= meth.diff.cutoff, 
                    pvalue < pvalue.cutoff), 
            aes(x = meth.diff, y = -log10(pvalue)),
            shape = 21, size = 4, color = "black", fill = "red"
        ) + geom_rug(
                data = res_promoters[res_promoters$symbol %in% genes,] %>% filter(
                    abs(meth.diff) >= meth.diff.cutoff, 
                    pvalue < pvalue.cutoff), 
                sides = "b", colour= 'black') + 
        scale_y_sqrt(limits=c(0,20))
    return (v)
}

In [ ]:
res_promoters[res_promoters$symbol %in% RNAme_hyper,] %>% filter(
                abs(meth.diff) >= meth.diff.cutoff, 
                pvalue < pvalue.cutoff)

In [ ]:
smaller_volcano(RNAme_hyper,'Hyper RNAme')

In [ ]:
res_promoters[res_promoters$symbol %in% RNAme_hypo,] %>% filter(
                abs(meth.diff) >= meth.diff.cutoff, 
                pvalue < pvalue.cutoff)

In [ ]:
# smaller_volcano(RNAme_hypo)

In [ ]:
            # geom_label_repel(
            #     data = res %>% filter(name %in% labels),
            #     size=2, 
            #     segment.colour = "black",
            #     aes(label = name), 
            #     color = 'grey10',fill='white',
            #     box.padding = 1, max.overlaps = Inf) + 
            # scale_color_manual(values = c("grey50", "green4")) +


### sensitivity

In [ ]:
smaller_volcano(resistance,'resistance hits')

## 

In [ ]:
data = myDiff_tiles_ann %>% mutate(
    label = case_when(
        myDiff_tiles_ann$symbol %in% RNAme_hyper ~ 'RNAme_hyper',
        myDiff_tiles_ann$symbol %in% RNAme_hypo ~ 'RNAme_hypo', 
        TRUE ~ ''
    )) 

In [ ]:
# rbind(
#     lowess(data$distance[data$label =='RNAme_hyper'],data$meth.diff[data$label =='RNAme_hyper'], f=.4) %>% data.frame %>% mutate(label='RNAme_hyper'),
#     lowess(data$distance[data$label ==''],data$meth.diff[data$label ==''], f=.01) %>% data.frame %>% mutate(label='')
# )
data %>%
    ggplot(aes(x=distance, y=meth.diff,fill=label)) + 
    geom_point(
        data=data, 
        aes(x=distance, y=meth.diff,fill='gray10',size=1),alpha=.01) + 
    geom_point(
        data=data %>% dplyr::filter(label %in% c('RNAme_hyper')), 
        aes(x=distance, y=meth.diff,colour=label,shape=label,size=1),alpha=.5) + 
    # geom_smooth(aes(colour=label), span = 10,method='lm') + 
    xlab("Distance to TSS") + 
    # ylim(-100,50) + 
    geom_hline(yintercept=0, linetype="dashed", color = "gray90", size=1) +
    xlim(-10**4*3,10**4*3) + 
    # facet_wrap(~label) +
    theme_Publication()    


In [ ]:
#      geom_point(data,
#         aes(x = meth.diff, y = -log10(pvalue),colour=label),
#         shape = 21, size = 3#, color = "black", fill = "red1"
#     # )# + geom_rug(data,
#     #     sides = "b", colour= 'red1'
#     # )
# # ggsave('diff-DNAme-volcano.pdf',width=8,height=7)

# res_promoters %>% dplyr::filter(pvalue < pvalue.cutoff, meth.diff > meth.diff.cutoff) %>% dplyr::select(symbol) %>% unique

# lowess(data$distance[keep], data$meth.diff[keep], f=0.1) 


# %>% filter(abs(meth.diff) >= meth.diff.cutoff, pvalue < pvalue.cutoff)

In [ ]:
plot_hyperhypo <- function (res, meth.diff.cutoff, pvalue.cutoff, title='', x_lim=FALSE,y_lim=NA,alpha = 5/10, size = 3) {
    if (x_lim==FALSE){
        x_min = res$meth.diff %>% min
        x_max = res$meth.diff %>% max
    } else if (length(x_lim)==2) {
        x_min = x_lim[1]
        x_max = x_lim[2]
    }
    colors_values = c("grey80", "#FF4500") # https://www.rapidtables.com/web/color/RGB_Color.html

    w <- wilcox.test(res$meth.diff, mu=-5, alternative = "less")
    t <- t.test(res$meth.diff, mu=-5, alternative = "less")
    
    res$sig <- as.factor(res$pvalue < pvalue.cutoff & abs(res$meth.diff) > meth.diff.cutoff)
    relevel(res$sig, ref=TRUE)
    
    vol <- res %>% ggplot(aes(x = meth.diff, y = -log10(pvalue))) + 
        geom_point(
            data = res %>% filter(
                abs(meth.diff) >= meth.diff.cutoff, 
                pvalue < pvalue.cutoff), 
            aes(x = meth.diff, y = -log10(pvalue)), 
            size = size, alpha = alpha, shape = 21, 
            color = colors_values[2],
            fill = colors_values[2]) + 

        geom_point(
            data = res %>% filter(
                (abs(meth.diff) < meth.diff.cutoff) | 
                (pvalue >= pvalue.cutoff)), 
            aes(x = meth.diff, y = -log10(pvalue)), 
            size = 1, alpha = alpha, 
            shape = 21, color = colors_values[1], 
            fill = colors_values[1]) + 
        
        geom_rug(
            data = res %>% filter(
                abs(meth.diff) >= meth.diff.cutoff, 
                pvalue < pvalue.cutoff), 
            alpha = alpha,
            sides = "b", colour= colors_values[2]) + 

        xlim(c(x_min, x_max)) + 
        ylim(c(0, y_lim)) + 
        # ylim(c(0, -log10(res %>% select(pvalue) %>% min ))) + 
        geom_hline(yintercept = -log10(pvalue.cutoff), linetype = "dashed", alpha = 5/10) + 
        ggtitle(title) +
        geom_text(aes(
            x_max*0.5,-log10(pvalue.cutoff)/1000,
            label = sprintf(
                "\n[mu=-5,alter=less]\nwilcox.test (-log10 p.value): %.2f\nt.test (-log10 p.value):%.2f",
                -log10(w$p.value), -log10(t$p.value)),
            vjust = x_min*0.05
        )) +
        theme(
            legend.position = "none"
        ) + 
        theme_Publication()
    return(vol)
}

In [ ]:
filtere_obj <- function(myobj){
    filtered.myobj=filterByCoverage(myobj,lo.count=10,lo.perc=NULL,
                                    hi.count=NULL,hi.perc=99.9)
    return(filtered.myobj)
    }

> ### Reading processed Bismark files 
> Bismark aligner scripts can produce per base methylation files. These files, namely "cytosine report"  and "coverage" files can be read in R using methylKit methRead() function. See Bismark manual for details on the files.


## CHG context

In [ ]:
meth_CHG = methylKit::processBismarkAln(
    files,
    sample.id=list('DMSO',"Decitabine"),
    assembly="hg38",
    save.folder='methylKit',
    save.context=NULL,
    read.context="CHG",
    nolap=FALSE,mincov=10,minqual=20,phred64=FALSE,
    treatment=c(0,1)
)

In [ ]:
saveRDS(meth_CHG, file = "methylKit/meth_CHG.rds")

## CHH context

In [ ]:
meth_CHH = methylKit::processBismarkAln(
    files,
    sample.id=list('DMSO',"Decitabine"),
    assembly="hg38",
    save.folder='methylKit',
    save.context=NULL,
    read.context="CHH",
    nolap=FALSE,mincov=10,minqual=20,phred64=FALSE,
    treatment=c(0,1)
)

In [ ]:
saveRDS(meth_CHH, file = "methylKit/meth_CHH.rds")

## CpG context

In [ ]:
# files = list.files('bismark/',pattern='srt.bam',full.names = TRUE) %>% as.list

In [ ]:
# meth = methylKit::processBismarkAln(
#     files,
#     sample.id=list('DMSO',"Decitabine"),
#     assembly="hg38",
#     save.folder='methylKit',
#     save.context=NULL,
#     read.context="CpG",
#     nolap=FALSE,mincov=10,minqual=20,phred64=FALSE,
#     treatment=c(0,1)
# )

In [ ]:
# saveRDS(meth, file = "methylKit/meth.rds")

In [ ]:
meth <- readRDS(file = "methylKit/meth.rds")

#### Descriptive statistics on samples

In [ ]:
pdf(file = "plots/MethylationStatsDMSO.pdf",   
    # The directory you want to save the file in
    width = 5, # The width of the plot in inches
    height = 5) # The height of the plot in inches

getMethylationStats(meth[[1]],plot=TRUE,both.strands=FALSE)

# Step 3: Run dev.off() to create the file!
dev.off()

In [ ]:
pdf(file = "plots/MethylationStatsDAC.pdf",   
    # The directory you want to save the file in
    width = 5, # The width of the plot in inches
    height = 5) # The height of the plot in inches

getMethylationStats(meth[[2]],plot=TRUE,both.strands=FALSE)

# Step 3: Run dev.off() to create the file!
dev.off()

#### Filtering samples based on read coverage

In [ ]:
meth1 = filtere_obj(meth)

#### Merging samples

In [ ]:
meth_merged=methylKit::unite(meth1, destrand=FALSE)

In [ ]:
pdf(file = "plots/MethylationCorrelation.pdf",   
    # The directory you want to save the file in
    width = 5, # The width of the plot in inches
    height = 5) # The height of the plot in inches

getCorrelation(meth_merged,plot=TRUE)

# Step 3: Run dev.off() to create the file!
dev.off()

### tileMethylCounts

In [ ]:
tiles = tileMethylCounts(meth,win.size=1000,step.size=1000,cov.bases=0)

In [ ]:
tiles_merged=methylKit::unite(tiles, destrand=FALSE)

In [ ]:
# tiles_merged %>% head

## Prepare genome annotations 
https://support.bioconductor.org/p/126514/

In [ ]:
# GTF = '~/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf'
BED12 = '~/genomes/hg38/gencode.v34/gencode.v34.annotation.bed'
# gtf <- rtracklayer::import(GTF)
# gene2name <- gtf[gtf$type == "gene"] %>% data.frame %>% 
#     column_to_rownames('gene_id') %>% 
#     dplyr::select('gene_name')

In [ ]:
gtf = gffToGRanges(BED12)

In [ ]:
rtracklayer::export.bed(myDiff_gr,con='delta_DNAme_table.bed')

In [ ]:
genes_ol <- findOverlaps(
    gtf[gtf$type == "gene"],
    myDiff_gr
)

In [ ]:
myDiff_gr[queryHits(gtf_ol)]

In [ ]:
myDiff_gtf_ann = cbind(
    gtf[queryHits(gtf_ol)] %>% data.frame,
    myDiff_gr[queryHits(gtf_ol)] %>% data.frame %>% dplyr::rename(cpg_start=start,cpg_end=end,cpg_chr=chr)
)

# myDiff_gtf_ann_gr = myDiff_tiles_ann %>% GRanges

In [ ]:
myDiff_gtf_ann

### Finding differentially methylated bases or regions

Download data from UCSC genome browser

https://nbis-workshop-epigenomics.readthedocs.io/en/latest/content/tutorials/methylationSeq/Seq_Tutorial.html#cpg-annotation

then, https://www.bioconductor.org/packages/devel/bioc/vignettes/methylKit/inst/doc/methylKit.html#4_Annotating_differentially_methylated_bases_or_regions

> https://www.nature.com/articles/s42003-021-01661-w#MOESM2

> Promoters are defined as 2 kb flanking the transcription start sites of genes (gencode basic). CGI
promoters (orange) were obtained by merging intervals of promoters with overlapping
CGIs (UCSC) while the residual promoters are defined as non-CGI promoters (violet).
CGIs not overlapping with any promoter regions were defined as orphan CGIs (yellow).
Exons (green) and introns (red) are obtained by subtracting CGI promoters and nonCGI promoters from gene exons and introns. All other regions are defined as intergenic
(blue).

In [ ]:
# First load the annotation data; i.e the coordinates of promoters, TSS, intron and exons
gene.obj=readTranscriptFeatures("genes.bed.gz")

In [ ]:
myDiff <- calculateDiffMeth(meth_merged,
                            overdispersion = "MN",
                            adjust="BH")

In [ ]:
# get hyper methylated bases and order by qvalue
myDiff25p.hyper <- getMethylDiff(myDiff,
                              difference=25,
                              qvalue=0.01,
                              type="hyper")
myDiff25p.hyper <- myDiff25p.hyper[order(myDiff25p.hyper$qvalue),]

# get hypo methylated bases and order by qvalue
myDiff25p.hypo <- getMethylDiff(myDiff,
                             difference=25,
                             qvalue=0.01,
                             type="hypo")
myDiff25p.hypo <- myDiff25p.hypo[order(myDiff25p.hypo$qvalue),]

# get all differentially methylated bases and order by qvalue
myDiff25p <- getMethylDiff(myDiff,
                        difference=25,
                        qvalue=0.01)
myDiff25p <- myDiff25p[order(myDiff25p$qvalue),]

In [ ]:
# Annotate hypermethylated CpGs ("target") with promoter/exon/intron
# information ("feature"). This function operates on GRanges objects, so we # first coerce the methylKit object to GRanges.
myDiff25p.anot <- annotateWithGeneParts(target = as(myDiff25p,"GRanges"),feature = gene.obj)

In [ ]:
# Summary of target set annotation
myDiff25p.anot

In [ ]:
# This can also be summarized for all differentially methylated CpGs
plotTargetAnnotation(myDiff25p.hyper.anot, main = "Differential Hypo Methylation Annotation")

### getAssociationWithTSS

In [ ]:
# View the distance to the nearest Transcription Start Site; the target.row column in the output indicates the row number in the initial target set
dist_tss <- getAssociationWithTSS(myDiff25p.anot)
# head(dist_tss)

In [ ]:
# See whether the differentially methylated CpGs are within promoters,introns or exons; the order is the same as the target set
# getMembers(myDiff25p.anot)

In [ ]:
# Load the CpG info
cpg_anot <- readFeatureFlank("CpG_islands.bed.gz", feature.flank.name = c("CpGi", "shores"), flank=2000)
diffCpGann <- annotateWithFeatureFlank(as(myDiff25p,"GRanges"), feature = cpg_anot$CpGi, 
                                       flank = cpg_anot$shores, feature.name = "CpGi", 
                                       flank.name = "shores")

In [ ]:
# # See whether the CpG in myDiff25p belong to a CpG Island or Shore
# head(getMembers(diffCpGann))

In [ ]:
# This can also be summarized for all differentially methylated CpGs
plotTargetAnnotation(diffCpGann, main = "Differential Methylation Annotation")

## Differentially Methylated Regions


In [ ]:
# Summarize the original object counts over a certain region, here the CpG Islands
# You can ignore the warnings here...
myobj_islands <- regionCounts(meth, cpg_anot$CpGi)

In [ ]:
# Filter the summarized counts by coverage
myobj_islands_filt <- filterByCoverage(myobj_islands,
                      lo.count=10,
                      lo.perc=NULL,
                      hi.count=NULL,
                      hi.perc=99.9)

In [ ]:
# Perform simple normalization
myobj_islands_filt_norm <- normalizeCoverage(myobj_islands_filt, method = "median")

In [ ]:
# # Merge the samples again
meth_islands <- methylKit::unite(myobj_islands_filt_norm, destrand=FALSE)

In [ ]:
# Test for differential methylation... This might take a few minutes.
myDiff_islands <- calculateDiffMeth(meth_islands)
# Rank by significance
myDiff_islands <- myDiff_islands[order(myDiff_islands$qvalue),]
# get all differentially methylated CpG Islands
myDiff_islands_25p <- getMethylDiff(myDiff_islands,difference=25,qvalue=0.01)

In [ ]:
myDiff_islands_25p_ann <- annotateWithGeneParts(as((myDiff_islands_25p), "GRanges"), gene.obj)

In [ ]:
myDiff_islands_ann <- annotateWithGeneParts(as((myDiff_islands), "GRanges"), gene.obj)

In [ ]:
# # View the distance to the nearest Transcription Start Site; the target.row column indicates the row number in myDiff_islands_25p
# head(getAssociationWithTSS(myDiff_islands_25p_ann))

In [ ]:
myDiff_islands_25p_ann <- annotateWithGeneParts(as((myDiff_islands_25p), "GRanges"), gene.obj)
# View the distance to the nearest Transcription Start Site; the target.row column indicates the row number in myDiff_islands_25p
head(getAssociationWithTSS(myDiff_islands_25p_ann))

In [ ]:
myDiff_islands %>% data.frame %>% plot_hyperhypo(5,0.05,title='myDiff_islands_25p') # +
    # scale_y_sqrt(limits=c(0,100))

In [ ]:
myDiff %>% as.data.frame %>%
    write.table(file = "delta_DNAme_bases_table.txt", sep = "\t",row.names = FALSE,quote=FALSE)

In [ ]:
myDiff_gr = myDiff %>% GRanges

In [ ]:
myDiff_tiles=calculateDiffMeth(tiles_merged)

### Annotating differential methylation events 
https://www.biostars.org/p/436554/

In [ ]:
# Gene annotation
meth_merged_ann = cbind(meth_merged,nearestTSS(meth_merged$chr, meth_merged$start) %>% rename(tss.strand=strand))

In [ ]:
# Gene annotation
myDiffTop_ann = cbind(myDiffTop,nearestTSS(myDiffTop$chr, myDiffTop$start) %>% rename(tss.strand=strand)) %>% dplyr::select(!c(gene_id,width))

In [ ]:
myDiffTop_ann %>% head

In [ ]:
tiles_merged_ann = cbind(tiles_merged,nearestTSS(tiles_merged$chr, tiles_merged$start) %>% rename(tss.strand=strand))

In [ ]:
# Gene annotation
myDiff_tiles_ann = cbind(myDiff_tiles,nearestTSS(myDiff_tiles$chr, myDiff_tiles$start) %>% rename(tss.strand=strand)) %>% dplyr::select(!c(gene_id,width))

In [ ]:
myDiff_tiles_ann %>% dplyr::filter(symbol %in% c('BCL2') & pvalue < 0.05)

In [ ]:
myDiff_tiles_ann %>% as.data.frame %>% 
    write.table(file = "delta_DNAme_table.txt", sep = "\t",row.names = FALSE,quote=FALSE)

In [ ]:
vol = myDiff_tiles_ann %>% plot_hyperhypo(5,0.01,title='Bisulfite-seq\nDecitabine vs. DMSO',x_lim=c(-110,110)) +
    scale_y_sqrt(limits=c(0,20))

In [ ]:
vol

In [ ]:
ggsave('plots/diff-DNAme-volcano.pdf',vol,width=8,height=7)

#### Finding differentially methylated promoters

In [ ]:
# PromCpG <- findOverlaps(
#     myDiff_tiles %>% GRanges,
#     myDiff_tiles_ann %>% dplyr::filter(distance > -2500 & distance < 2500) %>% GRanges
# )

In [ ]:
res_promoters = myDiff_tiles_ann %>% dplyr::filter(distance > -2500 & distance < 2500) #%>% GRanges

In [ ]:
# res_promoters %>% as.data.frame %>% 
#     write.table(file = "meth_prmoters_diff.txt", sep = "\t",row.names = FALSE,quote=FALSE)

In [ ]:
# res_promoters$symbol %>% unique %>% length == res_promoters$symbol %>% length

In [ ]:
# # labels = res_promoters[(abs(res_promoters$meth.diff) > 40) & (res_promoters$pvalue < 0.001),] %>% rownames

# res_promoters %>% plot_Volcano(20,0.05,title='CpG in Promoter\nDecitabine vs. DMSO') 

# #+ 
#         # geom_point(
#         #     data = res_promoters[labels,],
#         #     aes(x = meth.diff, y = -log10(pvalue)), 
#         #     shape = 21, size = 3, color = "grey40", fill = "red3") + 
#         # geom_label_repel(
#         #     data = res_promoters[labels,],
#         #     # data = res_promoters %>% filter(symbol %in% labels), 
#         #     aes(label = symbol), box.padding = 1, max.overlaps = Inf)

In [ ]:
res_promoters %>% plot_hyperhypo(5,0.05,title='TSS 2.5kb\nDecitabine vs. DMSO',x_lim=c(-110,110)) +
    scale_y_sqrt(limits=c(0,20))

In [ ]:
# myDiffTop_ann[myDiffTop_ann$symbol %in% RNAme_hypo,] %>% plot_hyperhypo(meth.diff.cutoff,pvalue.cutoff,title='Hypo RNAme')
# + 
    # geom_point(
    #     data = res_promoters %>% filter(
    #             abs(meth.diff) >= meth.diff.cutoff, 
    #             pvalue < pvalue.cutoff), 
    #     aes(x = meth.diff, y = -log10(pvalue)),
    #     shape = 21, size = 3, color = "black", fill = "red1"
    # ) + 
    #     geom_rug(
    #         data = res_promoters %>% filter(
    #             abs(meth.diff) >= meth.diff.cutoff, 
    #             pvalue < pvalue.cutoff), 
    #         sides = "b", colour= 'red1')

# ggsave('diff-DNAme-volcano.pdf',width=8,height=7)

#### Finding differentially methylated enhancers

In [ ]:
SupEnh = read.csv('SE_02_1028_SE_hg38.bed',sep='\t',header=T) %>% GRanges

SupEnh_ol <- findOverlaps(
    SupEnh,
    myDiff_tiles_ann %>% GRanges
)

myDiff_tiles_ann_gr = myDiff_tiles_ann %>% GRanges

In [ ]:
ls 

In [ ]:
SupEnhCpG = cbind(
    myDiff_tiles_ann_gr[queryHits(SupEnh_ol)] %>% data.frame, 
    SupEnh[queryHits(SupEnh_ol)] %>% data.frame %>% dplyr::rename(se_start=start,se_end=end,se_chr=chr)
)

In [ ]:
SupEnhCpG %>% dplyr::select(start,end,se_start,se_end)

#### Annotating differentially methylated bases or regions

In [ ]:
gene.obj = readTranscriptFeatures('methylKit/gene_annotation.bed')

In [ ]:
annotateWithGeneParts(as(myDiffTop,"GRanges"),gene.obj)

#### Regional analysis


> We can also summarize methylation information over a set of defined regions such as promoters or CpG islands. The function below summarizes the methylation information over a given set of promoter regions and outputs a methylRaw or methylRawList object depending on the input. We are using the output of genomation functions used above to provide the locations of promoters. For regional summary functions, we need to provide regions of interest as GRanges object.


In [ ]:
promoters=regionCounts(myobj,gene.obj$promoters)

head(promoters[[1]])


#### Convenience functions for annotation objects


In [ ]:
diffAnn=annotateWithGeneParts(as(myDiff25p,"GRanges"),gene.obj)

# target.row is the row number in myDiff25p
head(getAssociationWithTSS(diffAnn))


In [ ]:
getTargetAnnotationStats(diffAnn,percentage=TRUE,precedence=TRUE)


In [ ]:
plotTargetAnnotation(diffAnn,precedence=TRUE,
    main="differential methylation annotation")


In [ ]:
plotTargetAnnotation(diffCpGann,col=c("green","gray","white"),
       main="differential methylation annotation")


In [ ]:
getFeatsWithTargetsStats(diffAnn,percentage=TRUE)

In [ ]:
sm = ScoreMatrix(target = cage, windows = promoters)

heatMatrix(sm, xcoords = c(-1000, 1000))
plotMeta(sm, xcoords = c(-1000, 1000))


In [ ]:
myDiffTop_ann

In [ ]:
> sessionInfo()

# 

In [ ]:
!date